In [1]:
import moviepy.editor as mpy
import numpy as np
import pandas as pd
import os
import cv2
import re

In [2]:
def remove_outliers(src):
    num_labels,labels,stats,centroids = cv2.connectedComponentsWithStats(src,connectivity=4,ltype=None)
    img = np.zeros((src.shape[0],src.shape[1]),np.uint8) # create a black background of all 0
    for i in range(1,num_labels):
        mask = labels == i # this step is to determine the location of the area through labels, assign labels information to the mask array, and then use the mask array as the index of img array
        if stats[i][4] > 100: 
            img[mask] = 255 # areas larger than 100 shall be painted white, and areas smaller than 100 shall be painted black
        else:
            img[mask] = 0
    return img

In [3]:
def get_contours(frm):
    # convert frame to grayscale
    frm_gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)
    # remove tiny particels
    frm_out = remove_outliers(frm_gray)
    # convert frame to black and white
    frm_thresh = cv2.threshold(frm_out,0,255,cv2.THRESH_BINARY)[1]
    # extract contours
    contours,_ = cv2.findContours(frm_thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [4]:
def get_bbox_coords(clip, t=0):
    x_bbox=y_bbox=w_bbox=h_bbox=0
    # get the frame
    frm = clip.get_frame(t)
    # get the bounding box of the frame
    contours = get_contours(frm)
    # get the coordinates of the bounding box
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w > 100 and h > 100:
            x_bbox,y_bbox,w_bbox,h_bbox=x,y,w,h
    
    return x_bbox,y_bbox,w_bbox,h_bbox

In [5]:
def crop_and_resize(clip): 
    clip_x, clip_y = clip.size
    bbox = [clip_x,0,clip_y,0]
    average_h = 740
    
    # iterate through frames of the video clip
    for frm in clip.iter_frames():
        contours = get_contours(frm)
        
        # iterate over contours
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            # save rectangle that contains all boundingboxes inside it
            if x < bbox[0]:
                bbox[0] = x 
            if x+w > bbox[1]:
                bbox[1] = x+w 
            if y < bbox[2]:
                bbox[2] = y 
            if y+h > bbox[3]:
                bbox[3] = y+h 
    
    # crop and resize the clip
    _,_,_,clip_h = get_bbox_coords(clip)
    scaling_factor =  average_h/clip_h
    
    cropped_clip = clip.fx(mpy.vfx.crop, x1=bbox[0],x2=bbox[1],y1=bbox[2],y2=bbox[3])
    resized_clip = cropped_clip.resize(scaling_factor)
    
    #calculate the center of gravity of the human body in the resized clip
    x,y,w,h = get_bbox_coords(resized_clip)
    center = (round((2*x+w)/2),round((2*y+h)/2))
    
    return resized_clip, center

In [6]:
def change_dur(clip, duration):
    rate = clip.fps
    dur = clip.duration
    
    desired_rate = (rate*dur)/duration
    scaling_factor = desired_rate/rate

    # Modify the FPS
    clip = clip.set_fps(clip.fps*scaling_factor)
    # Apply speed up
    clip = clip.fx(mpy.vfx.speedx,scaling_factor)
    
    return clip

In [7]:
def fix_rotation(clip):
    if clip.rotation in (90,270):
            clip = clip.resize(clip.size[::-1])
            clip.rotation = 0
            
    return clip

In [8]:
in_dir = 'IndividualStimuli'
files = [os.path.join(in_dir, f) for f in os.listdir(in_dir)]

#out_dir = 'DyadStimuli'
#is_existend = os.path.exists(out_dir)
#if not is_existend:
#    os.mkdir(out_dir)
#    print(f'Directory "{out_dir}" created!')

file_pairings = []
all_pairings = []
possible_pairings = [[('gg09', 'gg19'), ('gg01', 'gg18'), ('gg10', 'gg11'), ('gg13', 'gg17'), ('gg08', 'gg20'), ('gg04', 'gg15'), ('gg02', 'gg06'), ('gg03', 'gg07')]]

#all_gestures_df = pd.read_excel('GesturesIndividualStimuli.xlsx', 'gestures_combinations')
#relevant_gestures_df = all_gestures_df.dropna()
#gesture_codes = relevant_gestures_df['GestureCode'].tolist()
#gesture_combis = [(gesture_codes[i],gesture_codes[j]) for i in range(0,len(gesture_codes)) for j in range(i,len(gesture_codes)) if i != j]

for gesture_combis in possible_pairings:
    for ind,comb in enumerate(gesture_combis):
        gestures_left = []
        gestures_right = []
        for f in files:
            if re.search(rf'({comb[0]})_(f01)', f):
                gestures_left.append(f)
            elif re.search(rf'({comb[1]})_(f02)', f):
                gestures_right.append(f)
        gesture_combis[ind] = [(left,right) for right in gestures_right for left in gestures_left if left != right]

In [9]:
for cnt,all_pairings in enumerate(possible_pairings):
    out_dir = f'DyadPairings_{cnt+1}'
    is_existend = os.path.exists(out_dir)
    if not is_existend:
        os.mkdir(out_dir)
        print(f'Directory "{out_dir}" created!')

    for pairing in all_pairings:
        for gestures in pairing:
            # Load the video files of the individual stimuli to be combined into a dyad
            gesture_left = mpy.VideoFileClip(gestures[0],has_mask=True)
            gesture_right = mpy.VideoFileClip(gestures[1],has_mask=True).fx(mpy.vfx.mirror_x)

            # Workaround for the moviepy bug causing videos with rotation metadata to be stretched
            gesture_left = fix_rotation(gesture_left)
            gesture_right = fix_rotation(gesture_right)

            # Transform the individual stimuli files
            gesture_left, center_left = crop_and_resize(gesture_left)
            gesture_right, center_right = crop_and_resize(gesture_right)

            # Speed up or slow down video to 3sec
            duration = 3
            gesture_right = change_dur(gesture_right,duration)
            gesture_left = change_dur(gesture_left,duration)

            # Load the background image
            background = mpy.ImageClip('background_dyads.png',duration=duration)

            # Create the video composition
            dyad = mpy.CompositeVideoClip([background, 
                                           gesture_left.set_position((660-center_left[0],background.size[1]-gesture_left.size[1]-170)), 
                                           gesture_right.set_position((1260-center_right[0],background.size[1]-gesture_right.size[1]-170))
                                          ])
            # Make the video grayscale
            dyad = dyad.fx(mpy.vfx.blackwhite)

            # Write the output video file
            out_path = os.path.join(out_dir,f'DS_l{gestures[0][21:-4]}_r{gestures[1][21:-4]}.mp4')
            dyad.write_videofile(out_path,fps=25,audio=False)

Directory "DyadPairings_1" created!
Moviepy - Building video DyadPairings_1/DS_lgg09_f01_rgg19_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg09_f01_rgg19_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg09_f01_rgg19_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg01_f01_rgg18_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg01_f01_rgg18_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg01_f01_rgg18_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg10_f01_rgg11_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg10_f01_rgg11_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg10_f01_rgg11_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg13_f01_rgg17_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg13_f01_rgg17_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg13_f01_rgg17_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg08_f01_rgg20_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg08_f01_rgg20_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg08_f01_rgg20_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg04_f01_rgg15_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg04_f01_rgg15_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg04_f01_rgg15_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg02_f01_rgg06_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg02_f01_rgg06_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg02_f01_rgg06_f02.mp4
Moviepy - Building video DyadPairings_1/DS_lgg03_f01_rgg07_f02.mp4.
Moviepy - Writing video DyadPairings_1/DS_lgg03_f01_rgg07_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadPairings_1/DS_lgg03_f01_rgg07_f02.mp4
